<a href="https://colab.research.google.com/github/sussahoo/DeepLearning/blob/master/CustomerSegmentation_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tags : ** **** 

**Customer segmentation, classification, sklearn- (one-hot-encode,StandardScaler/Normalize, label-encode, ColumnTransformer), keras, confusion matrix**

In [65]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/sussahoo/DeepLearning/master/data/Churn_Modelling.csv \
    -O /tmp/Churn_Modelling.csv

--2020-06-14 00:50:11--  https://raw.githubusercontent.com/sussahoo/DeepLearning/master/data/Churn_Modelling.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘/tmp/Churn_Modelling.csv’

/tmp/Churn_Modellin 100%[===================>] 668.81K  --.-KB/s    in 0.1s    

2020-06-14 00:50:11 (6.54 MB/s) - ‘/tmp/Churn_Modelling.csv’ saved [684858/684858]



In [70]:
# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Importing the libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('/tmp/Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].to_numpy()
y = dataset.iloc[:, 13].to_numpy()
print(type(X),X.shape,X.dtype)
print(X[:2,:])

<class 'numpy.ndarray'> (10000, 10) object
[[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]
 [608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]]




1.   Previously we used to label encode first and then one-hot, but with 0.20 one-hot-encode works with categorical strings.
2.   When we one-hot encode, we have 3 countires so 3 columns and 2 columns for 2 genders. so drop one from each.   

*Actually for geneder column, we don't need to one-hot-encode and drop first, just labelEncoder would have worked*



```
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
```











In [71]:
# Encoding categorical data, label encoder enocodes say 3 countries, to values 0,1,2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

"""
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
ct = ColumnTransformer(
    [('oh_enc', OneHotEncoder(sparse=False,drop='first')), [1]),],  # the column numbers I want to apply this to
    remainder='passthrough'  # This leaves the rest of my columns in place
)
"""
ct = ColumnTransformer(
    [('oh_enc', OneHotEncoder(sparse=False,drop='first'), [ 1,2]),],  # the column numbers I want to apply this to
    remainder='passthrough'  # This leaves the rest of my columns in place
)
X = ct.fit_transform(X)
print(X[:2,:])

[[0.0 0.0 0.0 619 42 2 0.0 1 1 1 101348.88]
 [0.0 1.0 0.0 608 41 1 83807.86 1 0 1 112542.58]]


In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [73]:
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Using TensorFlow backend.


Epoch 1/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.4855 - accuracy: 0.7955
Epoch 2/100
8000/8000 [==============================] - 1s 108us/step - loss: 0.4262 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.4208 - accuracy: 0.8006
Epoch 4/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4171 - accuracy: 0.8253
Epoch 5/100
8000/8000 [==============================] - 1s 107us/step - loss: 0.4148 - accuracy: 0.8288
Epoch 6/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.4139 - accuracy: 0.8320
Epoch 7/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4126 - accuracy: 0.8324
Epoch 8/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4112 - accuracy: 0.8321
Epoch 9/100
8000/8000 [==============================] - 1s 107us/step - loss: 0.4103 - accuracy: 0.8319
Epoch 10/100
8000/8000 [==============================]

In [75]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
print(new_prediction)

[[False]]


In [77]:
# Printing the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1538   57]
 [ 262  143]]
